## GPT 문장 생성으로 query-pos 튜닝 데이터셋 구축

### Asynchronous API 요청

In [ ]:
from openai import AsyncOpenAI
from pydantic import BaseModel
import pandas as pd
import json
import os
from tqdm.asyncio import tqdm_asyncio
import asyncio

class Triplet(BaseModel):
    id : int
    title : str
    query: str
    pos: list[str]

def dataset_to_df(dataset):
    df = pd.DataFrame(dataset)

    # df['title'] = df['query'].apply(lambda x : x['title'])
    # df['content'] = df['query'].apply(lambda x : x['content'])
    
    df['title'] = df.apply(lambda x : x['title'])
    df['content'] = df.apply(lambda x : x['content'])

    df.drop('query', axis=1, inplace=True)

    return df

async def process_row_async(row, instructions, client):
    id = row['id']
    title = row['title']
    query = row['content']
    prompt = f"\"id\" : {id}, \"title\" : {title}, \"query\" : \"{query}\""

    try:
        response = await client.responses.parse(
            model='gpt-4.1',
            instructions=instructions,
            input=[{"role": "user", "content": prompt}],
            text_format=Triplet,
        )
        return dict(response.output_parsed)
    except Exception as e:
        print(f"Error in row {title} : {e}")
        return {'id': id , 'title': title, 'query': query, 'pos': []}
    
async def generate_sentences_async(instrctions, client, dataset : pd.DataFrame, dir_path: str, concurrency: int = 6):
    sem = asyncio.Semaphore(concurrency) # API 동시 요청 수 제한

    async def sem_task(row):
        async with sem:
            return await process_row_async(row, instrctions, client)
        
    tasks = [sem_task(row) for _, row in dataset.iterrows()]
    results = []

    for coro in tqdm_asyncio.as_completed(tasks, total=len(tasks), desc="Processing"):
        result = await coro
        if result:
            results.append(result)

    os.makedirs(dir_path, exist_ok=True)
    path = os.path.join(dir_path, "relevant_incidents.jsonl")

    # 비동기 처리 결과값들 원래 쿼리 데이터 순서대로 정렬
    sorted_results = sorted(results, key=lambda x : x['id'])

    with open(path, 'w', encoding='utf-8') as f:
        for res in sorted_results:
            json.dump(res, f, ensure_ascii=False)
            f.write('\n')

### 형법.jsonl (= queries.jsonl) -> queries-edited.jsonl 

queries.jsonl : 형법.xlsx -> queries.jsonl

queries-edited.jsonl은 형법.jsonl 에서 아래 규칙을 적용하여,  
사람이 직접 검토 수정한 데이터. 코드로는 미구현됨   

1. 형법 제1편 제외, 기타 조항 제외.  

형에 대한 조항보다 무엇이 범법행위에 해당하는 지 위주로 진행  
id 472 부터 미수범, 예비음모 처리X (아예 제외)  
id 512 부터 병과 처리 X  
id 545 부터 상습범 처리 X  
형의 감경 처리 X  
세계주의 처리 X  
친족간의 범행 처리 X  

2. Query expansion  

-의 예에 의한다, 전항의 예에 따른다 등 참조표현 대체 및 정보추가.  

In [ ]:
from datasets import load_dataset
import nest_asyncio
import asyncio

client = AsyncOpenAI(
    api_key=os.environ.get("OPENAI_API_KEY")
)

dataset = load_dataset("json", data_files="../data/queries-edited.jsonl")['train'] # query expansion 된 데이터
ds_df = dataset_to_df(dataset)

ds_df[:30]

,id,title,content
0,219,제87조,제87조(내란) 대한민국 영토의 전부 또는 일부에서 국가권력을 배제하거나 국헌을 문...
1,220,제87조제1호,우두머리는 사형 무기징역 또는 무기금고에 처한다
2,221,제87조제2호,모의에 참여하거나 지휘하거나 그 밖의 중요한 임무에 종사한 자는 사형 무기 또는 5...
3,222,제87조제3호,부화수행(附和隨行)하거나 단순히 폭동에만 관여한 자는 5년 이하의 징역이나 금고에 처한다
4,223,제88조,제88조(내란목적의 살인) 대한민국 영토의 전부 또는 일부에서 국가권력을 배제하거나...
5,229,제91조제1호,헌법 또는 법률에 정한 절차에 의하지 아니하고 헌법 또는 법률의 기능을 소멸시키는 것
6,230,제91조제2호,헌법에 의하여 설치된 국가기관을 강압에 의하여 전복 또는 그 권능행사를 불가능하게 ...
7,231,제92조,제92조(외환유치) 외국과 통모하여 대한민국에 대하여 전단을 열게 하거나 외국인과 ...
8,232,제93조,제93조(여적) 적국과 합세하여 대한민국에 항적한 자는 사형에 처한다
9,234,제94조제1항,적국을 위하여 모병한 자는 사형 또는 무기징역에 처한다


문장 생성

In [ ]:
k = 5
with open("gpt_prompt.txt", "r", encoding="utf-8") as f:
    instructions = f.read().replace("#K#", str(k))

nest_asyncio.apply() # jupyter notebook 자체적인 running event loop 가 존재하므로 실행 중 루프 (주피터 노트북) 내 중첩된 루프를 허용

asyncio.run(generate_sentences_async(instructions, client, ds_df, dir_path="../data/gpt_output")) # -> relevant_incidents.jsonl

Processing: 100%|██████████| 363/363 [06:37<00:00,  1.09s/it]


### 기존 jsonl 파일 데이터 포맷 변경 (옵션)  

조항 구분 띄어쓰기 제거  

In [ ]:
from datasets import load_dataset
import os
import re


def trans_format(fname: str, law_kind: str):
    ds = load_dataset("json", data_files=fname)["train"]
    new_ds = ds.map(
        lambda x: {
            "id": x["id"],
            "title": f"{law_kind} {re.sub(r'(?<!^)(?=제)', ' ', x['query']['title'])}",  # 띄어쓰기 제거
            "content": x["query"]["content"],
        },
    )
    new_ds = new_ds.remove_columns(["query"])
    new_fname = os.path.splitext(fname)[0] + "_2" + ".jsonl"
    new_ds.to_json(new_fname, force_ascii=False)


trans_format("../data/queries-edited.jsonl", "형법")

Map:   0%|          | 0/762 [00:00<?, ? examples/s]

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Map:   0%|          | 0/363 [00:00<?, ? examples/s]

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]